In [ ]:
#  Most of these libraries are included in the Anaconda distribution of Python.  Download necessary libraries through the Anaconda prompt with pip install or conda install.  

import numpy as np
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from thermo.chemical import Chemical
import tkinter
from tkinter import *
from tkinter import filedialog
import os

In [ ]:
#  this cell opens a dialog to load the template spreadsheet

root = tkinter.Tk()
request_load_file = filedialog.askopenfile(initialdir=os.getcwd(),filetypes=[('Excel Files', '*.xlsx'),('CSV Files', '*.csv')])
if request_load_file:
      filepath = os.path.abspath(request_load_file.name)
root.destroy()
if filepath[-5:]=='.xlsx':
    df_inventory = pd.read_excel(filepath)
elif filepath[-4:]=='.csv':
    df_inventory = pd.read_csv(filepath)
df_inventory.head()

In [ ]:
#  this cell creates a temporary reference object for p codes and precautionary statements

result = requests.get(f'https://pubchem.ncbi.nlm.nih.gov/ghs/#_prec','lxml')
soup = BeautifulSoup(result.text,'lxml')

gross_precautions_list = list()
p_codes_list = list()
precaution_statements_list = list()

for i in soup.select('#pcode')[0].select('td'):
    gross_precautions_list.append(i.text)
for i in range(0,len(gross_precautions_list)):
    if not re.search(r'P\d\d\d',gross_precautions_list[i])==None:
        p_codes_list.append(gross_precautions_list[i])
for code in p_codes_list:
    precaution_statements_list.append(gross_precautions_list[gross_precautions_list.index(code)+1])
precaution_data_dict = {'P Codes':p_codes_list,'Precautionary Statements':precaution_statements_list}
df_precaution = pd.DataFrame(precaution_data_dict)

In [ ]:
#  Note: The loaded spreadsheet must contain the CAS numbers of the chemicals in a column named "CAS"
#  This cell creates new columns in the dataframe and cross-references PubChem IDs with given CAS numbers

df_inventory['PubChem ID']=np.nan
df_inventory['GHS Codes']=np.nan
df_inventory['Precautionary Statements']=np.nan
for i in df_inventory['CAS']:
    try:
        chem = Chemical(f'{i}')
        df_inventory.loc[df_inventory['CAS']==i,'PubChem ID'] = chem.PubChem
    except Exception:
        pass

In [ ]:
#  This cell will loop over the PubChem IDs and request GHS data from NCBI.  This could have a runtime of several minutes depending on your hardware and internet connection.

for chem_id in set(df_inventory['PubChem ID'].dropna()):
    result = requests.get(f'https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{int(chem_id)}/JSON/?response_type=display&heading=GHS%20Classification','lxml')
    soup = BeautifulSoup(result.text,'lxml').text
    if len(soup) > 90:
        pattern_hits = [m.start() for m in re.finditer(r'"H\d\d\d', soup)]
        ghs_codes_set = set()
        for i in range(0,len(pattern_hits)):
            phrase_start = pattern_hits[i]
            phrase_end = soup[phrase_start+1:].find('"')
            ghs_codes_set.add(soup[phrase_start+1:phrase_start+5])
        ghs_codes_list = list()
        for c in ghs_codes_set:
            ghs_start = soup.find(c)
            ghs_end = soup[soup.find(c)+1:].find('"')+1
            ghs_codes_list.append(soup[ghs_start:ghs_start+ghs_end])
        joined_ghs = ' --- '.join(ghs_codes_list)
        df_inventory.loc[df_inventory['PubChem ID']==chem_id,'GHS Codes'] = joined_ghs
        p_list = list()
        for i in set(soup[soup.find('Precautionary Statement Codes'):].replace('and ','').replace(' ','').replace('"\n}','').split(',')):
            if i in p_codes_list:
                p_list.append(i+' '+df_precaution['Precautionary Statements'][df_precaution[df_precaution['P Codes']==i].index[0]])
        joined_p = ' --- '.join(p_list)
        df_inventory.loc[df_inventory['PubChem ID']==chem_id,'Precautionary Statements'] = joined_p
    else:
        pass
df_inventory.head()

In [ ]:
#  If you prefer .csv format, simply replace '.xlsx' below with '.csv'
#  Edited spreadsheet will be saved in the same location as the origional template

df_inventory.to_excel(filepath[:filepath.rfind('.')]+'_ghs_edits.xlsx')